In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./datasets/email_spam.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2636 entries, 0 to 2635
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Kategori  2636 non-null   object
 1   Pesan     2636 non-null   object
dtypes: object(2)
memory usage: 41.3+ KB


In [9]:
stopwords = pd.read_csv('datasets/stopwords.csv', header=None, names=['words'])
stopwords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758 entries, 0 to 757
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   words   758 non-null    object
dtypes: object(1)
memory usage: 6.1+ KB
